# Load Teslafi data to MySQL using PySpark

In [ ]:
!echo $VIRTUAL_ENV
!env|grep SPARK
!echo $SPARK_HOME
# !pip install pandas findspark

In [ ]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
import pandas as pd
from configparser import ConfigParser

import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client.jar pyspark-shell'

import findspark
findspark.init()


In [ ]:
# !env|grep SPARK
# !cp -v /usr/share/java/mariadb-java-client.jar $SPARK_HOME/jars/

In [ ]:
# spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mariadb-java-client.jar").appName("TeslafiLoad").getOrCreate()
spark = SparkSession.builder.appName("TeslafiLoad").getOrCreate()
sc = spark.sparkContext


In [ ]:
# os.listdir(path='/data/data-files/teslafi/')

In [ ]:
# File location and type
csvdir = "/data/data-files/teslafi"
file_type = "csv"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# MySQL/MariaDB info
# Get credentials from file
config_section = 'csvload'
parser = ConfigParser()
_ = parser.read('CREDENTIALS.config')


user = parser.get(config_section, 'sqluser')
password = parser.get(config_section, 'sqlpassword')
host = parser.get(config_section, 'sqlhost')
port = parser.get(config_section, 'sqlport')
# db = parser.get(config_section, 'sqldb')
db = 'teslafi'
sqlurl = 'jdbc:mysql://' + host + ':' + port + '/' + db


# for path,name in [(f.path,f.name) for f in dbutils.fs.ls(csvdir) if f.path.endswith('.csv')  ]:
for fname in os.listdir(path=csvdir):
    if fname.endswith('.csv'):
        print(fname)
        # if not fname in ['ff_party.csv', 'ff_company.csv']:
        #     print('skipping ' + fname)
        #     continue
        df = spark.read.format('csv') \
        .option("inferSchema", infer_schema) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .load(csvdir + '/' + fname)
        # for a,b in df.dtypes:
            # if b == 'timestamp': 
            #     newdf = df.withColumn(a, df[a].cast(DateType()))
            #     print('   CHANGED column ' + a + ' to Date type')
            #     df = newdf

        df.write.format('jdbc').options(
            url=sqlurl,
            driver='org.mariadb.jdbc.Driver',
            dbtable='teslafi',
            user=user,
            password=password).mode('append').save()

In [ ]:
p = df.toPandas()

In [ ]:
p

In [ ]:
p['charger_phases'].unique()

In [ ]:
df.dtypes

In [ ]:
'ohai' == 'None'
